In [ ]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
import pickle as pkl

TRAINING = False


config_files_name = '../input/pytorch-all-mri-types-training/config.pkl'
a_file = open(config_files_name, "rb")
config = pkl.load(a_file)
a_file.close()

print(f'Best Weights Saved to {config_files_name}')
config

In [ ]:
if os.path.exists("../input/rsna-miccai-brain-tumor-radiogenomic-classification"):
    data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
    pytorch3dpath = "../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D"
else:
    data_directory = '/media/roland/data/kaggle/rsna-miccai-brain-tumor-radiogenomic-classification'
    pytorch3dpath = "EfficientNet-PyTorch-3D"
    
mri_types = ['FLAIR','T1w','T1wCE','T2w']

## U Can chosse larger parameters it's safe.
SIZE = config['SIZE']
NUM_IMAGES = config['NUM_IMAGES'] # NUM_IMAGES per Type
BS = config['BS'] # Batch Size
EPOCHS = config['EPOCHS']

SEED = config['SEED']

output_files_name = './bst_files.pkl'

sys.path.append(pytorch3dpath)
from efficientnet_pytorch_3d import EfficientNet3D

## Functions to load images

In [ ]:
def load_dicom_image(path, img_size=SIZE, voi_lut=True, rotate=0):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    if rotate > 0:
        rot_choices = [0, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
        data = cv2.rotate(data, rot_choices[rotate])
        
    data = cv2.resize(data, (img_size, img_size))
    return data


def load_dicom_images_3d(scan_id, mri_type="FLAIR", split="train", rotate=0, num_imgs=NUM_IMAGES, img_size=SIZE,):

    files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"), 
               key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f, rotate=rotate) for f in files[p1:p2]]).T 
    if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)
        
    if np.min(img3d) < np.max(img3d):
        img3d = img3d - np.min(img3d)
        img3d = img3d / np.max(img3d)
            
    return np.expand_dims(img3d,0)

a = load_dicom_images_3d("00000")
print(a.shape)
print(np.min(a), np.max(a), np.mean(a), np.median(a))

plt.imshow(a[0, :, :, 3]);

In [ ]:
# ALL MRI TYPES ONCE
def load_all_types_imgs_3d(scan_id, mri_types=mri_types, split="train", rotate=0, shuffle=True):
    imgs3d_ = [load_dicom_images_3d(scan_id=scan_id, mri_type=x, split=split, rotate=rotate) for x in mri_types]
    imgs3d = np.concatenate((imgs3d_), axis = -1)
    
    if shuffle:
        X = imgs3d
        #np.random.seed(SEED)
        return np.take(X, np.random.rand(X.shape[-1]).argsort(), axis=-1, out=X)
    
    return imgs3d

b = load_all_types_imgs_3d("00000", shuffle=True)
print(b.shape)
print(np.min(b), np.max(b), np.mean(b), np.median(b))

plt.imshow(b[0, :, :, 3]);

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(SEED)

## train / test splits

In [ ]:
train_df = pd.read_csv(f"{data_directory}/train_labels.csv")
display(train_df)


N_SPLITS = 10
skf = sk_model_selection.StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=54)

In [ ]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df["MGMT_value"])): 
    df_train, df_valid = train_df.iloc[trn_idx], train_df.iloc[val_idx]
    break

In [ ]:
train_df['MGMT_value'].value_counts()

## Model and training classes

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, label_smoothing=0.01, split="train", augment=False):
        self.paths = paths
        self.targets = targets
        self.label_smoothing = label_smoothing
        self.split = split
        self.augment = augment
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            #data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split=self.split)
            data = load_all_types_imgs_3d(str(scan_id).zfill(5), split=self.split)
        else:
            if self.augment:
                rotation = np.random.randint(0,4)
            else:
                rotation = 0

            #data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split="train", rotate=rotation)
            data = load_all_types_imgs_3d(str(scan_id).zfill(5), split="train", rotate=rotation)

        if self.targets is None:
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            y = torch.tensor(abs(self.targets[index]-self.label_smoothing), dtype=torch.float)
            return {"X": torch.tensor(data).float(), "y": y}


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out
    

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s            ",
                n_epoch, train_loss, train_time
            )
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            # if True:
            #if self.best_valid_score < valid_auc: 
            if self.best_valid_score > valid_loss: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_loss, self.lastmodel
                )
                self.best_valid_score = valid_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0

        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            
            loss = self.criterion(outputs, targets)
            loss.backward()

            sum_loss += loss.detach().item()

            self.optimizer.step()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)

                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)

                sum_loss += loss.detach().item()
                y_all.extend(batch["y"].tolist())
                outputs_all.extend(torch.sigmoid(outputs).tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        y_all = [1 if x > 0.5 else 0 for x in y_all]
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

## train models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_mri_type(df_train, df_valid, fold):
    #display(df_train.head())
    print(f'Training Fold-{fold}...')
    print(df_train.shape, df_valid.shape)

    train_data_retriever = Dataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        #df_train["MRI_Type"].values,
        augment=True
    )

    valid_data_retriever = Dataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        #df_valid["MRI_Type"].values
    )

    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=BS,
        shuffle=True,
        num_workers=8,pin_memory = True
    )

    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=BS//2,
        shuffle=False,
        num_workers=8,pin_memory = True
    )

    model = Model()
    model.to(device)


    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        EPOCHS, 
        train_loader, 
        valid_loader, 
        f"fold-{fold}-all_types", 
        6,
    )
    
    return trainer.lastmodel

In [ ]:
bst_files = {}

if TRAINING:
    for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df["MGMT_value"])): 
        df_train, df_valid = train_df.iloc[trn_idx], train_df.iloc[val_idx]
        bst_file = train_mri_type(df_train, df_valid, fold)
        bst_files[f'FOLD-{fold}'] = bst_file
        print(bst_file)

        break # Train only one fold 

In [ ]:
# save bst files
import pickle

a_file = open(output_files_name, "wb")
pickle.dump(bst_files, a_file)
a_file.close()

print(f'Best Weights Saved to {output_files_name}')

## Predict function

In [ ]:
def predict(modelfile, df, split):
    print("Predict:", modelfile, df.shape)
    data_retriever = Dataset(
        df.index.values, 
        #mri_type=df["MRI_Type"].values,
        split=split
    )

    data_loader = torch_data.DataLoader(
        data_retriever,
        batch_size=BS//2,
        shuffle=False,
        num_workers=8,
    )
   
    model = Model()
    model.to(device)
    
    checkpoint = torch.load(modelfile)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch["X"].to(device))).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            ids.extend(batch["id"].numpy().tolist())
            
    preddf = pd.DataFrame({"BraTS21ID": ids, "MGMT_value": y_pred}) 
    preddf = preddf.set_index("BraTS21ID")
    return preddf

## Ensemble for validation

In [ ]:
#output_path = './'
#modelfiles = glob.glob(output_path+'*pth')

#modelfiles

In [ ]:
output_files_name = '../input/pytorch-all-mri-types-training/bst_files.pkl'
a_file = open(output_files_name, "rb")
output = pickle.load(a_file)

modelfiles = list(output.values())
print(f'Best Weights Loaded from {output_files_name}')
modelfiles

In [ ]:
base_dir = '../input/pytorch-all-mri-types-training/'

In [ ]:
output

In [ ]:
for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df["MGMT_value"])): 
    if fold > 5:
        continue 
        
    df_train, df_valid = train_df.iloc[trn_idx], train_df.iloc[val_idx]
    df_valid = df_valid.set_index("BraTS21ID")
    df_valid["MGMT_pred"] = 0
    
    m = base_dir + output[f'FOLD-{fold}']
    pred = predict(m, df_valid, "train")
    df_valid["MGMT_pred"] += pred["MGMT_value"]

df_valid["MGMT_pred"] /= len(modelfiles)

auc = roc_auc_score(df_valid["MGMT_value"], df_valid["MGMT_pred"])
print(f"Validation ensemble AUC: {auc:.4f}")
sns.displot(df_valid["MGMT_pred"])

## Ensemble for submission

In [ ]:
submission = pd.read_csv(f"{data_directory}/sample_submission.csv", index_col="BraTS21ID")

submission["MGMT_value"] = 0
for m in modelfiles:
    m = base_dir + m 
    pred = predict(m, submission, split="test")
    submission["MGMT_value"] += pred["MGMT_value"]

submission["MGMT_value"] /= len(modelfiles)
submission["MGMT_value"].to_csv("submission.csv")

In [ ]:
submission

In [ ]:
sns.displot(submission["MGMT_value"])